In [6]:
from coffea.nanoevents import NanoEventsFactory
from coffea.nanoevents.schemas import BaseSchema
from config.selectionconfig import runsetting as rs

In [7]:
from config.selectionconfig import settings as cfg
import time

In [8]:
import dask_awkward as dak
import awkward as ak

In [9]:
from coffea.analysis_tools import PackedSelection

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
def measure_execution_time(func, *args, **kwargs):
    start_time = time.perf_counter()  # Start timing

    # Call the function with any provided arguments
    result = func(*args, **kwargs)

    end_time = time.perf_counter()  # End timing
    execution_time = end_time - start_time

    print(f"The function '{func.__name__}' took {execution_time} seconds to execute.")
    return result

In [36]:
start_time = time.perf_counter()
events = NanoEventsFactory.from_root(
        f"{rs.SINGLE_PATH}:Events",
        entry_stop=None,
        metadata={"dataset": "DYJets"},
        uproot_options={"timeout": 100},
        schemaclass=BaseSchema,
    ).events()
end_time = time.perf_counter()
execution_time = end_time - start_time
print(f"The function took {execution_time} seconds to execute.")

The function took 0.6647109169998657 seconds to execute.


In [28]:
measure_execution_time(uproot._dask.dask, f"{rs.SINGLE_PATH}:Events")

NameError: name 'uproot' is not defined

In [59]:
jetpt = events["Jet_pt"].compute()
jeteta = events["Jet_eta"].compute()
dak_zipped = dak.zip({"pt": dak.from_awkward(jetpt, npartitions=10),
                      "eta": dak.from_awkward(jeteta, npartitions=10)})

In [16]:
start_time = time.perf_counter()  # Start timing
(dak.num(dak_zipped)==2).compute()
end_time = time.perf_counter()
execution_time = end_time - start_time
print(f"The function took {execution_time} seconds to execute.")

The function took 0.02522283299981609 seconds to execute.


In [64]:
dak_zipped = dak_zipped[dak_zipped.pt>50]

In [65]:
dak_mask = dak.num(dak_zipped)>2

In [66]:
dak_zipped[dak_mask].compute()

<Array [[{pt: 195, eta: ..., ...}, ...], ...] type='8036 * var * {pt: float...'>

In [31]:
# correct way to compute events
events=events[dak_mask.compute()]

In [90]:
start_time = time.perf_counter()  # Start timing
(dak.num(events["Jet_pt"])==2).compute()
end_time = time.perf_counter()
execution_time = end_time - start_time
print(f"The function took {execution_time} seconds to execute.")

The function took 0.894880166000803 seconds to execute.


In [89]:
import operator as op
op.eq(dak.num(dak_zipped), 2)

dask.awkward<equal, npartitions=10>

In [2]:
lepsel = PackedSelection()

In [3]:
type(lepsel)

coffea.analysis_tools.PackedSelection